In [1]:
import neo
from neuralynxio import NeuralynxIO

dirname = r"C:\Users\Jeffe\OneDrive\Desktop\ncs"
io = NeuralynxIO(dirname=dirname)

# Read the block of data
block = io.read_block()
# Access segments and signals
for seg in block.segments:
    for signal in seg.analogsignals:
        print(signal)

[[ 2.38074905e+02  1.40018921e+02  9.17691574e+01 ...  5.76189461e+01
   1.20365005e+02  2.52113419e+02]
 [ 2.25165573e+02  1.03671379e+02  6.63167267e+01 ...  7.11691666e+01
   1.41972107e+02  2.35389267e+02]
 [ 2.36274307e+02  8.73134613e+01  5.10879860e+01 ...  8.36817551e+01
   1.69133591e+02  2.47932373e+02]
 ...
 [ 1.22074045e-01 -1.10477009e+01  8.85036755e+00 ...  1.99774170e+02
   2.04351944e+02 -7.19016113e+01]
 [-9.43021965e+00 -2.93893261e+01  3.38755465e+00 ...  1.70171219e+02
   1.60039062e+02 -6.05182076e+01]
 [ 7.23288679e+00 -1.97454758e+01 -9.91851616e+00 ...  1.94280838e+02
   1.74932098e+02 -2.24311047e+01]] uV


In [2]:
#View other properties of ncs files
for seg in block.segments:
    for signal in seg.analogsignals:
        print("Signal shape:", signal.shape)
        print("Sampling rate:", signal.sampling_rate)
        print("Units:", signal.units)
        print("Signal data (first 10 samples):", signal[:10])


Signal shape: (5579264, 13)
Sampling rate: 1600.0 Hz
Units: 1.0 uV
Signal data (first 10 samples): [[ 2.38074905e+02  1.40018921e+02  9.17691574e+01  6.01214647e+00
  -4.26038399e+01  2.44148090e-01 -4.76088753e+01  1.91045876e+01
   1.20731224e+02  8.45362701e+01  5.76189461e+01  1.20365005e+02
   2.52113419e+02]
 [ 2.25165573e+02  1.03671379e+02  6.63167267e+01 -2.34992528e+00
   2.01116982e+01  6.86666489e+01 -6.78731689e+01  1.16580706e+01
   1.13223671e+02  8.17896042e+01  7.11691666e+01  1.41972107e+02
   2.35389267e+02]
 [ 2.36274307e+02  8.73134613e+01  5.10879860e+01  4.54725790e+00
   9.85747910e+00  6.28070946e+01 -6.95822067e+01 -2.19733286e+00
   1.09866638e+02  8.10571671e+01  8.36817551e+01  1.69133591e+02
   2.47932373e+02]
 [ 2.19611206e+02  5.72222061e+01  4.48011742e+01 -1.48014774e+01
   1.52897739e+01  7.15964279e+01 -6.36616135e+01 -9.82696056e+00
   8.49635315e+01  5.51164284e+01  1.00527977e+02  1.74199661e+02
   2.35145126e+02]
 [ 2.15216537e+02  3.95214691e+01

In [ ]:
import numpy as np

In [13]:
#concatenate
final_list = []
for i in range(13):
    combined_data2 = np.column_stack((all_signals.T[i], timestamps))
    final_list.append(combined_data2)

# Print the first 10 samples of the concatenated data
print("Combined data (first 10 samples):")
print(final_list[:10])


Combined data (first 10 samples):
[array([[ 2.38074905e+02,  0.00000000e+00],
       [ 2.25165573e+02,  6.25000000e-04],
       [ 2.36274307e+02,  1.25000000e-03],
       ...,
       [ 1.22074045e-01,  3.48703813e+03],
       [-9.43021965e+00,  3.48703875e+03],
       [ 7.23288679e+00,  3.48703937e+03]]) * dimensionless, array([[ 1.40018921e+02,  0.00000000e+00],
       [ 1.03671379e+02,  6.25000000e-04],
       [ 8.73134613e+01,  1.25000000e-03],
       ...,
       [-1.10477009e+01,  3.48703813e+03],
       [-2.93893261e+01,  3.48703875e+03],
       [-1.97454758e+01,  3.48703937e+03]]) * dimensionless, array([[ 9.17691574e+01,  0.00000000e+00],
       [ 6.63167267e+01,  6.25000000e-04],
       [ 5.10879860e+01,  1.25000000e-03],
       ...,
       [ 8.85036755e+00,  3.48703813e+03],
       [ 3.38755465e+00,  3.48703875e+03],
       [-9.91851616e+00,  3.48703937e+03]]) * dimensionless, array([[ 6.01214647e+00,  0.00000000e+00],
       [-2.34992528e+00,  6.25000000e-04],
       [ 4.5472

In [11]:
import json
from pint import UnitRegistry

# Initialize a unit registry
ureg = UnitRegistry()
Q_ = ureg.Quantity
#dict of everything
data = {
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list), "dimensionless")]
}
def custom_serializer(obj):
    if isinstance(obj, ureg.Quantity):
        return {"magnitude": obj.magnitude.tolist() if isinstance(obj.magnitude, np.ndarray) else obj.magnitude,
                "units": str(obj.units)}
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

# Convert dictionary to JSON string
with open("data.json", "w") as json_file:
    json.dump(data, json_file, default=custom_serializer, indent=4)


In [12]:
# check the results
def custom_decoder(dct):
    if "magnitude" in dct and "units" in dct:
        magnitude = np.array(dct["magnitude"]) if isinstance(dct["magnitude"], list) else dct["magnitude"]
        return Q_(magnitude, dct["units"])
    return dct

with open("data.json", "r") as json_file:
    data_dict = json.load(json_file, object_hook=custom_decoder)

print(data_dict["Sample_Frequency"], type(data_dict["Sample_Frequency"]))
print(data_dict["Unit"], type(data_dict["Unit"]))
print(data_dict["Data"][0], type(data_dict["Data"][0]))

1600 hertz <class 'pint.Quantity'>
1 microvolt <class 'pint.Quantity'>
[[[238.0749053955078 0.0]  [225.1655731201172 0.000625]  [236.27430725097656 0.00125]  ...  [0.12207404524087906 3487.038125]  [-9.430219650268555 3487.03875]  [7.232886791229248 3487.039375]] [[140.0189208984375 0.0]  [103.67137908935547 0.000625]  [87.31346130371094 0.00125]  ...  [-11.047700881958008 3487.038125]  [-29.389326095581055 3487.03875]  [-19.74547576904297 3487.039375]] [[91.76915740966797 0.0]  [66.31672668457031 0.000625]  [51.08798599243164 0.00125]  ...  [8.850367546081543 3487.038125]  [3.38755464553833 3487.03875]  [-9.918516159057617 3487.039375]] ... [[57.61894607543945 0.0]  [71.1691665649414 0.000625]  [83.68175506591797 0.00125]  ...  [199.774169921875 3487.038125]  [170.1712188720703 3487.03875]  [194.2808380126953 3487.039375]] [[120.36500549316406 0.0]  [141.97210693359375 0.000625]  [169.1335906982422 0.00125]  ...  [204.35194396972656 3487.038125]  [160.0390625 3487.03875]  [174.9320983

In [15]:
# write each cell data to json - first one
data1 = {
    "Number": "CSCI_1",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[0]), "dimensionless")]
}
# Convert dictionary to JSON string
with open("CSCI1.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

In [16]:
# write each cell data to json - Second one, and so on
data2 = {
    "Number": "CSCI_2",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[1]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI2.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data4 = {
    "Number": "CSCI_4",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[2]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI4.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data5 = {
    "Number": "CSCI_5",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[3]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI5.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data6 = {
    "Number": "CSCI_6",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[4]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI6.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data7 = {
    "Number": "CSCI_7",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[5]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI7.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data8 = {
    "Number": "CSCI_8",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[6]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI8.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data9 = {
    "Number": "CSCI_9",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[7]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI9.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data10 = {
    "Number": "CSCI_10",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[8]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI10.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data11 = {
    "Number": "CSCI_11",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[9]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI11.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data13 = {
    "Number": "CSCI_13",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[10]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI13.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data15 = {
    "Number": "CSCI_15",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[11]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI15.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

data16 = {
    "Number": "CSCI_16",
    "Sample_Frequency": Q_(1600, "Hz"),
    "Unit": Q_(1, "uV"),
    "Data": [Q_(np.array(final_list[12]), "dimensionless")]
}

# Convert dictionary to JSON string
with open("CSCI16.json", "w") as json_file:
    json.dump(data1, json_file, default=custom_serializer, indent=4)

In [18]:
#ntt files
io = NeuralynxIO(dirname=r"C:\Users\Jeffe\OneDrive\Desktop\ntt")


# Read the block of data
block = io.read_block()
# Access segments and signals


In [32]:
import quantities as pq
def custom_serializer(obj):
    if isinstance(obj, pq.Quantity):
        return {
            "magnitude": obj.magnitude.tolist() if isinstance(obj.magnitude, np.ndarray) else obj.magnitude,
            "units": str(obj.units)
        }
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

In [60]:


checklist = [4,14,24,30,46,53,57,62]
numberlist=["TT2","TT5","TT6","TT8","TT9","TT10","TT11","TT15"]
count=0
data_list=[]
for seg in block.segments:
    for i, spiketrain in enumerate(seg.spiketrains):
        print(f"Tetrode {i} - Number of spikes:", len(spiketrain))
        print(f"Tetrode {i} - Spike times (first 10):", spiketrain.times[:10])
        print(f"Tetrode {i} - Units:", spiketrain.units)
        print(f"Tetrode {i} - Sampling Rate:", spiketrain.sampling_rate)
        print(f"Tetrode {i} - Annotations:", spiketrain.annotations)
        data_list.append(spiketrain.times)
        print(len(data_list[-1]))
        if i in checklist:

            data_dict = {
                "Number": numberlist[count],
                "Sample_Frequency": spiketrain.sampling_rate,
                "Unit": "s",
                "Data": data_list
                        }
            # Convert dictionary to JSON string
            with open(numberlist[count]+".json", "w") as json_file:
                json.dump(data_dict, json_file, default=custom_serializer, indent=4)

            count=count+1
            data_list=[]
            


Tetrode 0 - Number of spikes: 237220
Tetrode 0 - Spike times (first 10): [0.011437 0.023531 0.024406 0.028156 0.032906 0.046    0.054125 0.073125
 0.080312 0.099562] s
Tetrode 0 - Units: 1.0 s
Tetrode 0 - Sampling Rate: 32000.0 Hz
Tetrode 0 - Annotations: {'id': '0'}
237220
Tetrode 1 - Number of spikes: 2487
Tetrode 1 - Spike times (first 10): [ 1.211312  3.400812  3.707625  4.788    14.334375 14.614781 17.604781
 19.243968 25.279906 31.089312] s
Tetrode 1 - Units: 1.0 s
Tetrode 1 - Sampling Rate: 32000.0 Hz
Tetrode 1 - Annotations: {'id': '1'}
2487
Tetrode 2 - Number of spikes: 184418
Tetrode 2 - Spike times (first 10): [0.002812 0.020718 0.025343 0.043406 0.047187 0.109468 0.148718 0.160375
 0.211156 0.220937] s
Tetrode 2 - Units: 1.0 s
Tetrode 2 - Sampling Rate: 32000.0 Hz
Tetrode 2 - Annotations: {'id': '2'}
184418
Tetrode 3 - Number of spikes: 35538
Tetrode 3 - Spike times (first 10): [0.20725  0.383031 0.424468 0.509562 0.6665   0.749562 0.936125 1.031468
 1.038156 1.042656] s
Te

In [65]:
with open('TT2.json', 'r') as json_file:
    data = json.load(json_file)

# Print the JSON data to check the results
print("Sample Frequency:", data['Sample_Frequency'])
print("Unit:", data['Unit'])
print("First spike train (limited view):", data["Data"][4])


Sample Frequency: {'magnitude': 32000.0, 'units': '1.0 Hz'}
Unit: s
First spike train (limited view): {'magnitude': [14.817375000000538, 24.98165599999993, 40.120875000000524, 61.06446800000049, 61.0685310000008, 61.22140600000057, 61.22362500000054, 64.58056200000101, 64.60468700000092, 65.83565600000111, 66.12724999999955, 66.16453100000035, 66.83040600000095, 66.84996800000044, 67.31671800000004, 67.34128100000089, 69.61996800000088, 75.68703099999948, 97.34387500000048, 98.2247810000008, 98.59515600000123, 117.86612500000047, 124.55515600000035, 166.03865600000063, 173.78781200000049, 195.56118700000115, 226.85446799999954, 244.27934300000015, 248.9564680000003, 248.95900000000074, 281.1017810000012, 287.22843699999976, 291.9142810000012, 300.21590600000127, 300.4883750000008, 320.4353119999996, 336.0715930000006, 349.76640600000064, 349.7923119999996, 349.8965619999999, 359.8523430000005, 363.1350000000002, 363.39187500000116, 364.54565600000024, 378.5885620000008, 378.99196800000